In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"incals","key":"4f0043b4971092eab2b54bb5b32c6710"}'}

In [0]:
!cp kaggle.json ~/.kaggle/


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c champs-scalar-coupling

 76% 34.0M/44.5M [00:00<00:00, 61.6MB/s]
100% 44.5M/44.5M [00:00<00:00, 112MB/s] 
 82% 11.0M/13.4M [00:00<00:00, 58.4MB/s]
100% 13.4M/13.4M [00:00<00:00, 65.4MB/s]
 95% 60.0M/62.9M [00:00<00:00, 52.7MB/s]
100% 62.9M/62.9M [00:00<00:00, 116MB/s] 
 74% 36.0M/48.7M [00:00<00:00, 54.9MB/s]
100% 48.7M/48.7M [00:00<00:00, 70.9MB/s]
 90% 78.0M/86.6M [00:00<00:00, 55.7MB/s]
100% 86.6M/86.6M [00:01<00:00, 88.8MB/s]
  0% 0.00/5.31M [00:00<?, ?B/s]
100% 5.31M/5.31M [00:00<00:00, 48.1MB/s]
  0% 0.00/497k [00:00<?, ?B/s]
100% 497k/497k [00:00<00:00, 156MB/s]
 87% 8.00M/9.24M [00:00<00:00, 82.1MB/s]
100% 9.24M/9.24M [00:00<00:00, 84.1MB/s]
 76% 38.0M/49.8M [00:00<00:00, 59.5MB/s]
100% 49.8M/49.8M [00:00<00:00, 86.2MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 136MB/s]


In [0]:
#import librarys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
!unzip train.csv.zip
!unzip test.csv.zip

#who are you bro ?
#what is combo?

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [13]:
!unzip structures.csv.zip

Archive:  structures.csv.zip
  inflating: structures.csv          


In [0]:
structures = pd.read_csv('structures.csv')

def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train_df = map_atom_info(train_df, 0)
train_df = map_atom_info(train_df, 1)

test_df = map_atom_info(test_df, 0)
test_df = map_atom_info(test_df, 1)

In [16]:
!unzip scalar_coupling_contributions.csv.zip
scalar_df = pd.read_csv('scalar_coupling_contributions.csv')
scalar_df.tail()

Archive:  scalar_coupling_contributions.csv.zip
  inflating: scalar_coupling_contributions.csv  


,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
4658142,dsgdb9nsd_133884,17,4,2JHC,3.586440,0.019741,0.150477,-0.213205
4658143,dsgdb9nsd_133884,17,5,3JHC,0.674583,-0.007276,0.305078,-0.403388
4658144,dsgdb9nsd_133884,17,6,3JHC,1.337470,-0.028423,0.312240,-0.447920
4658145,dsgdb9nsd_133884,17,7,2JHC,4.800620,0.139202,-0.053102,-0.124710
4658146,dsgdb9nsd_133884,17,8,1JHC,115.975000,0.378277,0.450213,1.130600


In [18]:
#replace atom_0 and atom_1 
#C:0,H:1,N:2

train_df=train_df.replace({'atom_0': {'H': 0, 'C': 1,'N':2}})
train_df=train_df.replace({'atom_1': {'H': 0, 'C': 1,'N':2}})

#test set

test_df=test_df.replace({'atom_0': {'H': 0, 'C': 1,'N':2}})
test_df=test_df.replace({'atom_1': {'H': 0, 'C': 1,'N':2}})



#replace type with 
# '2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN:6', '2JHN':7

train_df=train_df.replace({'type': {'2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN':6, '2JHN':7}})
test_df=test_df.replace({'type': {'2JHC':0, '1JHC':1, '3JHH':2, '3JHC':3, '2JHH':4, '1JHN':5, '3JHN':6, '2JHN':7}})


TypeError: ignored

In [19]:
molecules_list = test_df['molecule_name']
molecules_list.head(2)

0    dsgdb9nsd_000004
1    dsgdb9nsd_000004
Name: molecule_name, dtype: object

In [20]:
train_df = train_df.drop('molecule_name',axis=1)
train_df = train_df.drop('id',axis=1)

test_df =  test_df.drop('molecule_name',axis=1)
test_df = test_df.drop('id',axis=1)

train_df.head()

,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,1,0,1,84.8076,0,0.002150,-0.006031,0.001976,1,-0.012698,1.085804,0.008001
1,1,2,4,-11.2570,0,0.002150,-0.006031,0.001976,0,1.011731,1.463751,0.000277
2,1,3,4,-11.2548,0,0.002150,-0.006031,0.001976,0,-0.540815,1.447527,-0.876644
3,1,4,4,-11.2543,0,0.002150,-0.006031,0.001976,0,-0.523814,1.437933,0.906397
4,2,0,1,84.8074,0,1.011731,1.463751,0.000277,1,-0.012698,1.085804,0.008001


In [0]:
#taking y value out from train dataframes
y_df = train_df['scalar_coupling_constant']

#removing scalar_coupling_constant from train dataframes
train_df= train_df.drop('scalar_coupling_constant',axis=1)

In [22]:
#convert into numpy array
x_train = np.array(train_df,dtype='float32')
y_train = np.array(y_df,dtype='float32')
x_test = np.array(test_df,dtype='float32')

x_train.shape,y_train.shape,x_test.shape

((4658147, 11), (4658147,), (2505542, 11))

In [0]:
#spilit data for validation and cross check only for 20%

from sklearn.model_selection import train_test_split
(x_train,x_validate,y_train,y_validate) = train_test_split(
    x_train,y_train,test_size=0.2,random_state=12345
)

In [24]:
#normalise the data

from sklearn import preprocessing

x_train = preprocessing.scale(x_train)
x_validate = preprocessing.scale(x_validate)
x_test = preprocessing.scale(x_test)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:189: UserWarning: Numerical issues were enc

In [0]:
#models

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier


In [26]:
#error occured Unknown label type: 'continous'
#solution found from https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_validate_encoded = lab_enc.fit_transform(y_validate)
print(y_train_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(y_train_encoded))

[ 986490  958513  562677 ...  776977 1009294  998621]
continuous
multiclass
multiclass


In [0]:
#1 model logistic regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train_encoded)
y_pred_lr = logreg.predict(x_test)
print('Training Accuracy for logistic regression is : { } '.format(logreg.score(x_train, y_train_encoded)))
print('Testing Accuracy for logistic regression is : { } '.format(logreg.score(x_validate, y_validate_encoded)))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
